In [1]:
import librosa
import naplib
import soundfile
import os
import pandas as pd
import numpy as np

## Word audibility using the spectrogram

In [4]:
import logging

logging.getLogger("naplib").setLevel(logging.FATAL)



targets_complete = pd.read_csv('target_distractor_final_final.csv', sep = ";")

directory = './word_seg_targets_final/'

sent_id = []
words = []
start = []
end = []
word_order = []

##Load the word onsets
for root,dirs,files in os.walk(directory):
    for file in files:
        if file.endswith(".csv"):
            if file[:3] in list(targets_complete['target'].str[:3]):

                p = os.path.join(directory, file)
                f=pd.read_csv(p, sep = ";")
                
                ##Remove silence in the beginning and end
                f = f.drop(f.loc[(f.MAU == "(...)")].index)
                f = f.drop(f.loc[f.TOKEN == -1].index)
        
                words_sent = list(f['TOKEN'].unique())
                wo = 1
                
                for w in words_sent:
                    f_w = f.loc[f['TOKEN'] == w]
                    word_start = f_w.iloc[0,0]
                    word_end = f_w.iloc[-1,0] + f_w.iloc[-1,1]
                    word = f_w.iloc[0,5]
        
                    word_order.append(wo)
                    wo = wo + 1
                    
                    sent_id.append(file)
                    words.append(word)
                    start.append(word_start)
                    end.append(word_end)

sent_words = pd.DataFrame(zip(sent_id, words, start, end, word_order), columns = ['id', 'word', 'start', 'end', 'word_order'])

##Calculate seconds
sent_words['start_s'] = sent_words['start']/44100
sent_words['end_s'] = sent_words['end']/44100

aud_list = []

for s_i in sent_words['id'].unique():
    if s_i[0:3] in list(targets_complete['target'].str[:3]):
        s_df = sent_words[sent_words['id'] == s_i]
    
        audio_path = "../behavior_stimuli/targets_final_norm/"
        audio_path_dis = "../behavior_stimuli/distractors_new_clean/"
        dis_col = targets_complete[targets_complete['target'].str[:3] == s_i[0:3]]
    
        dis = dis_col.iloc[0,1]
        fpath_dis = audio_path_dis + dis
        
        y_dis,sr = soundfile.read(fpath_dis)
    
        fpath = audio_path + s_i[:3] + ".wav"
    
        y,sr = soundfile.read(fpath)
    
        y = y*(pow(10,-10/20))
        for i,r in s_df.iterrows():
            start_dis = int(44100/2) + r['start']
            end_dis = int(44100/2) + r['end']
            
            y_cut_dis = y_dis[start_dis:end_dis]
        
            y_cut = y[r['start']:r['end']]
    
            spx = naplib.features.auditory_spectrogram(y_cut, sr)
    
            spy = naplib.features.auditory_spectrogram(y_cut_dis, sr)
    
            glimpsed = abs(spx > spy*(pow(10,-4/20)))
            ratio = np.count_nonzero(glimpsed)/np.count_nonzero(spx)
            aud_list.append(ratio)


sent_words['audibility'] = aud_list

df_audibility = sent_words.copy()
df_audibility['ids'] = df_audibility['id'].str[0:3]

df_audibility['filename'] =  df_audibility['id'].str[0:8]

df_audibility = df_audibility.drop(['id', 'start', 'end'], axis=1)

df_audibility.to_csv('./trf_input/audibility_words_final.csv')

In [5]:
targets_complete = pd.read_csv('target_distractor_final_final.csv', sep = ";")

directory = './word_seg_targets_final/'


sent_id = []
words = []
phones = []
start = []
end = []
word_order = []
wordnum = []
syl_nums = []
syls = []

for root,dirs,files in os.walk(directory):
    for file in files:
        if file[:3] in list(targets_complete['target'].str[:3]):

            p = os.path.join(directory, file)
            if p.endswith(".csv"):
                f=pd.read_csv(p, sep = ";")
            
                ##Remove silence in the beginning and end
                f = f.drop(f.loc[(f.MAU == "(...)")].index)
                f = f.drop(f.loc[f.TOKEN == -1].index)
                f = f.reset_index(drop=True)
                wo = 1
        
                syl_add = ""
                syl_num = 1
                syl_count = 1
                
                for i,p in f.iterrows():
                    f_w = p
                    phone_start = p.iloc[0]
                    phone_end = p.iloc[0] + p.iloc[1]
                    phone = p.iloc[3]
                    word = p.iloc[5]
                    numw = int(p.iloc[2])+1
        
                    word_order.append(wo)
                    wo = wo + 1
        
    
        
                    
                    sent_id.append(file)
                    phones.append(phone)
                    words.append(word)
                    wordnum.append(numw)
        
                    start.append(phone_start)
                    end.append(phone_end)
                


sent_words = pd.DataFrame(zip(sent_id, words, phones, start, end, word_order, wordnum), columns = ['id', 'words', 'phone', 'start', 'end', 'phone_order', 'wordnum'])

##Calculate seconds
sent_words['start_s'] = sent_words['start']/44100
sent_words['end_s'] = sent_words['end']/44100

In [6]:
import logging

logging.getLogger("naplib").setLevel(logging.FATAL)



aud_list = []

phone_aud_upsampled = []


for s_i in sent_words['id'].unique():
    if s_i[0:3] in list(targets_complete['target'].str[:3]):

        matches = []
        aud_trial = []
        up = []
        
        s_df = sent_words[sent_words['id'] == s_i]
    
        audio_path = "../behavior_stimuli/targets_final_norm/"
        audio_path_dis = "../behavior_stimuli/distractors_new_clean/"
        dis_col = targets_complete[targets_complete['target'].str[:3] == s_i[0:3]]
    
        dis = dis_col.iloc[0,1]
        fpath_dis = audio_path_dis + dis
        
        y_dis,sr = soundfile.read(fpath_dis)
    
        fpath = audio_path + s_i[:3] + ".wav"
    
        y,sr = soundfile.read(fpath)
    
        y = y*(pow(10,-10/20))
    
    
        for i,r in s_df.iterrows():
            start_dis = int(44100/2) + r['start']
            end_dis = int(44100/2) + r['end']
            
            y_cut_dis = y_dis[start_dis:end_dis]
        
            y_cut = y[r['start']:r['end']]
    
            spx = naplib.features.auditory_spectrogram(y_cut, sr)
    
            spy = naplib.features.auditory_spectrogram(y_cut_dis, sr)
            
            glimpsed = abs(spx > spy*(pow(10,-4/20)))
            ratio = np.count_nonzero(glimpsed)/np.count_nonzero(spx)
            aud_list.append(ratio)
    
                            


sent_words['audibility'] = aud_list



df_audibility_phone = sent_words.copy()

df_audibility_phone['ids'] = df_audibility_phone['id'].str[0:3]

df_audibility_phone['filename'] =  df_audibility_phone['id'].str[0:8]

df_audibility_phone = df_audibility_phone.drop(['id', 'start', 'end'], axis=1)


df_audibility_phone.to_csv('./trf_input/audibility_phones_final.csv')